In [ ]:
import requests, pandas as pd, time
from newspaper import Article
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

BASE_URL = "https://www.hankyung.com/article/"
HEADERS = {
    "User-Agent": ("Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
                   "AppleWebKit/537.36 (KHTML, like Gecko) "
                   "Chrome/127.0.0.0 Safari/537.36"),
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8",
    "Referer": "https://www.hankyung.com/",
}

def crawl_url(url):
    # 단일 URL 크롤링: requests로 가져와 newspaper 파싱 (403시 /amp 폴백)
    try:
        r = requests.get(url, headers=HEADERS, timeout=12)
        if r.status_code == 403:
            r = requests.get(url.rstrip("/") + "/amp", headers=HEADERS, timeout=12)
        if r.status_code in (403, 404):
            return None
        r.raise_for_status()

        art = Article(url, language='ko')
        art.set_html(r.text)
        art.parse()
        if not art.title or not art.text:
            return None
        return {"url": url, "title": art.title, "text": art.text, "publish_date": art.publish_date}
    except:
        return None

def crawl_g_articles(date_str, max_workers=31):
    """g 기사 크롤링 (0000~9999)"""
    urls = [f"{BASE_URL}{date_str}{num:04}g" for num in range(10000)]
    results = []

    with ThreadPoolExecutor(max_workers=max_workers) as ex:
        futures = [ex.submit(crawl_url, u) for u in urls]
        collected = 0
        with tqdm(total=len(futures), desc=f"g {date_str}") as pbar:
            for fut in as_completed(futures):
                res = fut.result()
                if res:
                    results.append(res)
                    collected += 1
                pbar.set_postfix_str(f"수집 {collected}") # 현재 수집 건수
                pbar.update(1)

                time.sleep(0.06)  # 너무 빠른 요청 방지(차단 완화)

    df = pd.DataFrame(results).drop_duplicates(subset=["url"])
    tqdm.write(f"최종 수집 기사 수: {len(df)}") # 최종 수집 건 수
    return df

In [2]:
day_str = "20250819"

In [ ]:
out_df = crawl_g_articles(day_str)
out_df.to_csv(f"/Users/leesangwon/Documents/ThemeStock_file/Hankyung_news/hankyung_g_{day_str}.csv", index=False)

out_df